# Topic Modeling Lab

### To do list:
   
- To-do:
    - interpreting topics (out of notebook placeholder)
    - top topics by personal attributes
    - comparing LDA & NMF topics (deal with alignment)
    - credit to https://medium.com/@aneesha/topic-modeling-with-scikit-learn-e80d33668730


# 0. Setup
### Step 1: Import the packages we'll use

In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage
from scipy.interpolate import griddata
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.preprocessing import normalize
from bs4 import BeautifulSoup
import warnings
from matplotlib import pyplot as plt
%matplotlib inline

### Step 2: Read in our data 

In [ ]:
profiles = pd.read_csv('data/clean_profiles.tsv', sep='\t')
profiles.head(2)

### Step 3: Pick which section of the profiles you want to analyze.
#### Options:
- text - All of the text from a profile
- essay0 - My self summary
- essay1 - What I’m doing with my life
- essay2 - I’m really good at
- essay3 - The first thing people usually notice about me
- essay4 - Favorite books, movies, show, music, and food
- essay5 - The six things I could never do without
- essay6 - I spend a lot of time thinking about
- essay7 - On a typical Friday night I am
- essay8 - The most private thing I am willing to admit
- essay9 - You should message me if...

#### Replace `'essay0'` in the cell below with the essay you want to look at.

In [ ]:
profile_section_to_use = 'essay0'

### Step 4: Clean up the text for that essay.
#### Helper function for cleaning up text
- removes HTML code, link artefacts
- converts to lowercase

In [ ]:
# Some of the essays have just a link in the text. BeautifulSoup sees that and gets 
# the wrong idea. This line hides those warnings.
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

def clean(text):
    if pd.isnull(text):
        t = np.nan
    else:
        t = BeautifulSoup(text, 'lxml').get_text()
        t = t.lower()

        bad_words = ['http', 'www', '\nnan']

        for b in bad_words:
            t = t.replace(b, '')
    if t == '':
        t = np.nan
    
    return t

#### Clean and select the text.

In [ ]:
print('Cleaning up profile text for', profile_section_to_use, '...')
profiles['clean'] = profiles[profile_section_to_use].apply(clean)

print('We started with', profiles.shape[0], 'profiles.')
print("Dropping profiles that didn't fill out the essay we chose...")
profiles.dropna(axis=0, subset=['clean'], inplace=True)

print('We have', profiles.shape[0], 'profiles left.')

# 1. Topic Modeling
#### Some parameters: change these to get different numbers of topics or words per topic

In [ ]:
#how many topics we want our model to find
ntopics = 20

#how many top words we want to display for each topic
nshow = 10

#what we will use as our documents, here the cleaned up text of each profile
documents = profiles['clean'].values

## 1.1 LDA
### Step 1: Convert text to numbers the computer understands
- LDA takes "count vectors" as input, that is, a count of how many times each word shows up in each document. 
    - Here we tell it to only use the 1,000 most popular words, ignoring stop words
- [Learn more](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) about LDA

In [ ]:
tf_vectorizer = CountVectorizer(max_features=1000, stop_words='english')

print("Vectorizing text by word counts...")
tf_text = tf_vectorizer.fit_transform(documents)

tmp = tf_text.get_shape()
print("Our transformed text has", tmp[0], "rows and", tmp[1], "columns.")

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()

print("The first few words (alphabetically) are:\n", tf_feature_names[:20])

### Step 2: Build a topic model using LDA

- LDA can be a little slow. We'll use a faster method later on.
- Set `n_jobs=` to the number of processors you want to use to compute LDA. If you set it to `-1`, it will use all available processors. 

In [ ]:
model = LatentDirichletAllocation(n_components=ntopics, max_iter=10, 
                                  learning_method='online', n_jobs=-1)

print('Performing LDA on vectors...')
lda = model.fit(tf_text)

print('Done!')

#### Functions to sort and show us the most important words in each topic

In [ ]:
def describe_topic(topic, feature_names, n_words=10):
    words = []
    # sort the words in the topic by importance
    topic = topic.argsort() 
    # select the n_words most important words
    topic = topic[:-n_words - 1:-1]
    # for each important word, get it's name (i.e. the word) from our list of names
    for i in topic:
        words.append(feature_names[i])
    # print the topic number and its most important words, separated by spaces
    return " ".join(words)

def display_topics(components, feature_names, n_words=10):
    # loop through each topic (component) in the model; show its top words
    for topic_idx, topic in enumerate(components):
        print("Topic {}:".format(topic_idx), describe_topic(topic, feature_names, n_words))
    return

def sort_topics(components):
    # run hierarchical clustering to find related groups of topics
    l = linkage(components, "ward")
    # calculate the id of the final cluster
    last_id = 2 * ntopics - 2
    # start with the final cluster and break it into smaller clusters
    order = [last_id]
    for i, row in reversed(list(enumerate(l))):
        # find the current cluster in the list and break it into two smaller clusters
        cluster_id = ntopics + i
        index = order.index(cluster_id)
        order = order[:index] + [row[0], row[1]] + order[(index+1):]
    # sort topics by the order calcuated above and return a copy
    components = [x[1] for x in sorted(zip(order, components))]
    return np.array(components)

#### Functions to compare different topics to each other

In [ ]:
def get_similarity(a, b=None):
    # normalize the topics so that their dot product is 1
    a = normalize(a)
    # if only one set of topics is given, compare it to itself
    if b is None:
        b = a
    else:
        b = normalize(b)
    # create a 2-D array to store the results of the similarity calcluation
    topic_similarity = np.zeros((ntopics, ntopics))
    # loop through each topic in both a and b
    for topic_idx, row in enumerate(a):
        for topic_jdx, col in enumerate(b):
            # calculate the similarity using a dot product
            topic_similarity[topic_idx, topic_jdx] = np.inner(row, col)
    return topic_similarity

def describe_intersection(components, a, b, feature_names, n_words=10):
    # normalize
    topic_a = components[a,:] / components[a,:].sum()
    topic_b = components[b,:] / components[b,:].sum()
    # multiply components of a and b to highlight words common in both
    x = topic_a * topic_b
    print("Words in {} and {}:".format(a, b), describe_topic(x, feature_names, n_words))

def describe_difference(components, a, b, feature_names, n_words=10):
    # normalize
    topic_a = components[a,:] / components[a,:].sum()
    topic_b = components[b,:] / components[b,:].sum()
    # multiply components of a with complement of components in b
    x = topic_a * (1 - topic_b)
    print("Words in {} but not in {}:".format(a, b), describe_topic(x, feature_names, n_words))

def plot_topics(components):
    # sort topics into similar groups
    components = sort_topics(components)
    # calculate similarity between topics
    topic_similarity = get_similarity(components)
    # create a figure and plot the similarites
    plt.figure()
    fig, ax = plt.subplots(figsize=(8,8))
    plt.imshow(np.log10(topic_similarity), cmap='Blues')
    # move the ticks to the top and maker sure each topic is labeled
    ax.xaxis.tick_top()
    plt.xticks(range(20)); plt.yticks(range(20))
    # plot a colorbar legend
    plt.colorbar()
    
def plot_confusion(x, y):
    # normalize all topics so their inner product is 1
    a = normalize(y)
    b = normalize(x)
    # match topics in b with their most similar topic in a
    sorted_b = []
    order_b = []
    idx_b = np.arange(ntopics)
    for ta in a:
        # of the b topics not yet assigned, find the one that best matches
        best_b = max([(np.inner(ta, b[i,:]), i) for i in range(b.shape[0])])[1]
        # move the b topic into the sorted list
        sorted_b.append(b[best_b,:])
        order_b.append(idx_b[best_b])
        b = np.delete(b, best_b, axis=0)
        idx_b = np.delete(idx_b, best_b, axis=0)
    # replace b topics with the sorted version
    b = np.array(sorted_b)
    # find similarity
    topic_similarity = get_similarity(a, b)
    # create a figure and plot the similarity valuse
    plt.figure()
    fig, ax = plt.subplots(figsize=(8,8))
    plt.imshow(np.log10(topic_similarity), cmap='Blues')
    # move the ticks to the top and reorder the x ticks because b was sorted
    ax.xaxis.tick_top()
    plt.xticks(range(20), order_b)
    plt.yticks(range(20))
    # show a colorbar legend
    plt.colorbar()

### Step 3: Show our topics with the top words in each

In [ ]:
lda_topics = sort_topics(lda.components_)
display_topics(lda_topics, tf_feature_names, n_words=nshow)

### Step 4: Compare topics to each other

We can compare topics visually by plotting the similarity of each topic to each other topic.
Groups of similar topics are placed closer together.
- Why are the diagonal cells the darkest?
- What do dark or light bands represent?
- Do you see dark regions made of several cells? If so, what do they represent?

In [ ]:
plot_topics(lda_topics)

#### Examine the words that make two topics similar or different
We can also compare two topics to each other by looking at words that are common in both,
or words that are common in one but not the other.
Try changing `topic_a` and `topic_b` to different topic numbers.

In [ ]:
topic_a = 0
topic_b = 3

describe_intersection(lda_topics, topic_a, topic_b, tf_feature_names, n_words=nshow)
describe_difference(lda_topics, topic_a, topic_b, tf_feature_names, n_words=nshow)
describe_difference(lda_topics, topic_b, topic_a, tf_feature_names, n_words=nshow)

### Step 5: Interpret these topics
- This part is for you to do: code can't do it for you.
- Look at the list of important words for each topic, and think about these questions.
    - What do the words have in common?
    - What could someone write that would use most of those words?
    - What does this topic seem to be about?
- Try to come up with a short, catchy name for each topic.
    - For example, if the words were "san francisco city moved living born years raised lived live", you might call it "places lived" because the topic seems to be about where people currently live (San Francisco) and where they were born / raised / moved from. 
- Try other numbers of topics.
    - If the topics seem repetitive, you might want to try looking for fewer topics.
    - If the topics seem confusing or vague, you might want to try looking for more topics (so that they can be more specific).

## 1.2 NMF
### Step 1: Convert text to numbers the computer understands
- NMF takes "tf-idf vectors" as input. Tf-idf stands for "text frequency - inverse document frequency." 
    - Text frequency is the same as the count vectors above: how often does each word appear in the text. 
    - Inverse document frequency means we divide ("inverse") by the number of documents the word is in. (If everyone uses the word, it isn't very helpful for figuring out what different people are talking about. So this measurement looks for words that are used a lot in some documents, and not at all in others.)
    - Here we tell it to only use the 1,000 most popular words, ignoring stop words
- [Learn more](https://en.wikipedia.org/wiki/Non-negative_matrix_factorization#Text_mining) about NMF.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

print("Vectorizing text by TF-IDF...")
tfidf_text = tfidf_vectorizer.fit_transform(documents)

tmp = tfidf_text.get_shape()
print("Our transformed text has", tmp[0], "rows and", tmp[1], "columns.")

#### The features are the same, because they are just the list of words in the text

In [ ]:
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print("The first few words (alphabetically) are:\n", tfidf_feature_names[:20])

### Step 2: Build a topic model using NMF

- NMF is faster than LDA and often works a little better for small documents like we have here.

In [ ]:
model = NMF(n_components=ntopics, alpha=.1, l1_ratio=.5, init='nndsvd')

print('Performing NMF on vectors...')
nmf = model.fit(tfidf_text)

print('Done!')

### Step 3: Show our topics with the top words in each

In [ ]:
nmf_topics = sort_topics(nmf.components_)
display_topics(nmf_topics, tfidf_feature_names, nshow)

### Step 4: Compare topics to each other
We can compare topics visually by plotting the similarity of each topic to each other topic. Groups of similar topics are placed closer together because we sorted them above.

In [ ]:
plot_topics(nmf_topics)

#### Examine the words that make two topics similar or different
We can also compare two topics to each other by looking at words that are common in both, or words that are common in one but not the other. Try changing topic_a and topic_b to different topic numbers.

In [ ]:
topic_a = 0
topic_b = 1

describe_intersection(nmf_topics, topic_a, topic_b, tfidf_feature_names, n_words=nshow)
describe_difference(nmf_topics, topic_a, topic_b, tfidf_feature_names, n_words=nshow)
describe_difference(nmf_topics, topic_b, topic_a, tfidf_feature_names, n_words=nshow)

### Step 5: Interpret these topics
- This part is for you to do: code can't do it for you.
- Look at the list of important words for each topic, and think about these questions.
    - What do the words have in common?
    - What could someone write that would use most of those words?
    - What does this topic seem to be about?
- Try to come up with a short, catchy name for each topic.
    - For example, if the words were "san francisco city moved living born years raised lived live", you might call it "places lived" because the topic seems to be about where people currently live (San Francisco) and where they were born / raised / moved from. 
- Try other numbers of topics.
    - If the topics seem repetitive, you might want to try looking for fewer topics.
    - If the topics seem confusing or vague, you might want to try looking for more topics (so that they can be more specific).

### Step 6: Compare the topics from LDA and NMF
We can compare the topics visually using a confusion matrix plot.
The NMF topics are along the X axis and the LDA are along the Y axis.
The NMF topics are sorted to match the closest LDA topic.

In [ ]:
plot_confusion(x=nmf_topics, y=lda_topics)

By looking at the LDA and NMF topic words and the confusion matrix, consider the following questions:
- Do any of the topics seem to be the same in both models?
- Are some topics in one model but not the other?
- Do the topics you get from one of the models make more sense than the ones you get from the other?


### Step 7: Compare profiles based on their topics
For each profile, we calculate how strongly each topic appears.
This code uses the NMF model.
To use LDA instead, remove the `#` at the beginning of the second line.

In [ ]:
model, text = nmf, tfidf_text
#model, text = lda, tf_text
model_doc_topic = model.transform(text)

#### Functions to visualize and compare topics

In [ ]:
def visualize_profile(model_doc_topic, profile_id):
    # plot a stem diagram for a single profile
    plt.figure(figsize=(8,4))
    plt.xticks(range(ntopics))
    plt.xlabel('Topic number')
    plt.ylabel('Proportion of profile about this topic')
    plt.stem(model_doc_topic[profile_id,:])
    
def profile_histogram(model_doc_topic, topic_id):
    # get profile values within a single topic
    values = model_doc_topic[:,topic_id]
    # calculate logarithmic bins based on smallest nonzero value
    bins = np.logspace(np.log10(min([v for v in values if v > 0])),np.log10(0.4),50)
    # plot the histogram
    plt.figure(figsize=(8,4))
    plt.ylabel('Number of profiles')
    plt.xlabel('Amount of topic {} in profile'.format(topic_id))
    plt.hist(values, bins=bins)
    plt.gca().set_xscale('log')

def profile_scatter(model_doc_topic, topic_x_id, topic_y_id):
    # create a scatter plot of profiles with each axis representing one topic
    plt.figure(figsize=(8,8))
    plt.loglog(model_doc_topic[:,topic_x_id], model_doc_topic[:,topic_y_id], '.', markersize=1)
    plt.xlabel('Amount of topic {}'.format(topic_x_id))
    plt.ylabel('Amount of topic {}'.format(topic_y_id))

#### Plot the topic breakdown of particular profiles
Examine different profiles by changing the value of the `profile_id` varible.

In [ ]:
visualize_profile(model_doc_topic, profile_id=0)

#### Visualize all of the profiles by their topic content
We can choose a topic and plot a histogram of that topic's relevance to each of the profiles.
- Try different topics by changing `topic_id`.
- Are there any topics with multiple peaks? What does that mean?

In [ ]:
profile_histogram(model_doc_topic, topic_id=1)

In [ ]:
profile_histogram(model_doc_topic, topic_id=5)

#### Analyze the relationship between two topics using a scatter plot of all profiles
- Try different topic combinations.
- Do the profiles cluster into distinct groups?
- What are the pros and cons of using a 2-D scatter plot vs the histogram above?

In [ ]:
profile_scatter(model_doc_topic, topic_x_id=0, topic_y_id=1)

### Going Further
Here are some ideas for doing a more detailed exploration of the data.
- Are certain topics correlated with age, sex, education, or other demographic data?
- Are any topics opposites of each other?
- Visualize profiles for different demographics in different colors.
- What are the profiles that most exemplify each topic?